## Demo #3: SQL Data Processing Capabilities

### 1. Install necessary dependencies

In [ ]:
#r "nuget:Microsoft.SqlToolsService,*-*"

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"

### 2. Connect to the Azure SQL database

In [ ]:
#!connect mssql --kernel-name AdventureWorks "Persist Security Info=False; Integrated Security=true; Initial Catalog=meetupdate-2023; User ID=vojtechkozuch;Password=a@fq@gQkVuDZE4o!N4UC8Dwz; Trusted_Connection=false; Server=interactive-data.database.windows.net" --create-dbcontext

### 3. Fetch city addresses based on user input

In [ ]:
var city = await GetInputAsync("Enter city name: ");

#### Option no. 1: Query the data directly using SQL

In [ ]:
#!sql-AdventureWorks --name cityQueryResults
#!set --value @csharp:city --name city

SELECT AddressLine1, City, PostalCode 
FROM SalesLT.Address
WHERE City = @city;

#### Option no. 2: Consume provided DB context in C#

In [ ]:
class AddressDto 
{
    public string AddressLine1 { get; set; }
    public string City { get; set; }
    public string PostalCode { get; set; }
}

var cityAddresses = await AdventureWorks.Addresses
                                        .Where(a => a.City == city)
                                        .Select(a => new AddressDto
                                        {
                                            AddressLine1 = a.AddressLine1,
                                            City = a.City,
                                            PostalCode = a.PostalCode
                                        })
                                        .ToListAsync();

cityAddresses

#### Appropriate data visualization in HTML table

In [ ]:
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Address Table</title>
    <style>
        table {
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
        }
        
        th, td {
            border: 1px solid #ccc;
            padding: 10px;
            text-align: left;
        }

        th {
            background-color: #f2f2f2;
            font-weight: bold;
        }

        tr:nth-child(even) {
            background-color: #f2f2f2;
        }
    </style>
</head>
<body>
    <table id="addressTable">
        <thead>
            <tr>
                <th>AddressLine1</th>
                <th>City</th>
                <th>PostalCode</th>
            </tr>
        </thead>
        <tbody>
        </tbody>
        </tbody>
    </table>
</body>

In [ ]:
// #!share --from csharp cityAddresses
#!share --from sql-AdventureWorks cityQueryResults

function populateTable(data) {
    const tableBody = document.getElementById('addressTable').getElementsByTagName('tbody')[0];

    data.forEach((item) => {
        const newRow = tableBody.insertRow();

        const addressLine1Cell = newRow.insertCell(0);
        addressLine1Cell.textContent = item.AddressLine1;

        const cityCell = newRow.insertCell(1);
        cityCell.textContent = item.City;

        const postalCodeCell = newRow.insertCell(2);
        postalCodeCell.textContent = item.PostalCode;
    });
}

// Populate the table with address data
//populateTable(cityAddresses);
populateTable(cityQueryResults[0].data);

### 4. Query all addresses, group them by city and visualize

In [ ]:
#!sql-AdventureWorks --name groupedCitiesQueryResults

SELECT City, COUNT(*) AS AddressesCount 
FROM SalesLT.Address 
GROUP BY City 
HAVING COUNT(*) > 10

#### Load Plotly.js using RequireJS

In [ ]:
plotlyloader = (require.config({
    paths: {
        d3: 'https://cdn.jsdelivr.net/npm/d3@7.4.4/dist/d3.min',
        jquery: 'https://cdn.jsdelivr.net/npm/jquery@3.6.0/dist/jquery.min',
        plotly: 'https://cdn.plot.ly/plotly-2.14.0.min'
    },

    shim: {
        plotly: {
            deps: ['d3', 'jquery'],
            exports: 'plotly'
        }
    }
}) || require);

#### Prepare HTML layout

In [ ]:
<div id='cities-pie-chart' style='height:400px'></div>

#### Populate obtained SQL data from JavaScript

In [ ]:
#!share --from sql-AdventureWorks groupedCitiesQueryResults

plotlyloader(['d3', 'plotly'], function (d3, plotly) {
    var trace1 = {
        labels: groupedCitiesQueryResults[0].data.map(d => d.City),
        values: groupedCitiesQueryResults[0].data.map(d => d.AddressesCount),
        type: 'pie',
        marker: {
            colors: ['rgb(177, 127, 38)', 'rgb(124, 103, 37)', 'rgb(99, 79, 37)', 'rgb(129, 180, 179)']
        }
      };
          
      var data = [trace1];
      plotly.newPlot('cities-pie-chart', data);
  });